# Random Forest à la main

## Importation des bibliothèques

In [81]:
import numpy as np
import pandas as pd
import random
from collections import Counter
from datetime import datetime
        


## Charger le fichier

In [82]:
# Charger le fichier Excel
df = pd.read_excel("C:\\Users\\anton\\Downloads\\sncf_data.xlsx")  

# Convertir en CSV
df.to_csv("data_csv", index=False, encoding="utf-8")

# Vérifier les premières lignes du fichier
print("Aperçu des données :")
print(df.head())


Aperçu des données :
     symbol supplier serial letter  repair complexity (0NA)  \
0  79544181     SCLE            AB                        1   
1  79402561     SCLE            BA                        0   
2  79540265     SCLE            BA                        0   
3  79540265     SCLE            BB                        0   
4  79540265     SCLE           BBM                        0   

   evaluation ratio between repair cost and cost of new (0NA)  \
0                                                  1            
1                                                  0            
2                                                  0            
3                                                  0            
4                                                  0            

   \nexisting substitute product (0NA)  \
0                                    1   
1                                    0   
2                                    1   
3                                    1   

## Conversion des données

In [83]:
# On remplace les cellules vides par la valeur 0
df.fillna(0, inplace=True)  # Remplace les NaN par 0

# Convertir les colonnes de dates en nombre de jours depuis aujourd'hui
for col in df.select_dtypes(include=['datetime64']):
    df[col] = (datetime.today() - df[col]).dt.days

df.head()

,symbol,supplier,serial letter,repair complexity (0NA),evaluation ratio between repair cost and cost of new (0NA),\nexisting substitute product (0NA),component having a role in security (0NA),does the product/block if the component have a role in security? (0NA),several components involved? (0NA),is the product recent? (<10 years) (0NA),...,do we have equipment to carry out approval tests? (0NA),have we done a technical validation? (0NA),generally at least equivalent criterion,active quantity,quantity of existing stock at SNCF?,qté cible,"no longer supply (new or used), no longer supply new, reparable, supply new",product policy (short-term vision year - 2023),processing date,label
0,79544181,SCLE,AB,1,1,1,1,1,1,0,...,0,0,1,1050,837,0.0,supply new,6,602,stock
1,79402561,SCLE,BA,0,0,0,0,0,1,0,...,1,1,1,436,999,0.0,supply new,100,596,redesign mineur
2,79540265,SCLE,BA,0,0,1,0,1,0,0,...,1,1,1,70,1655,0.0,supply new,100,940,substitution
3,79540265,SCLE,BB,0,0,1,0,1,0,0,...,1,1,1,149,1655,0.0,supply new,100,940,substitution
4,79540265,SCLE,BBM,0,0,1,0,1,0,0,...,1,1,1,24,1655,0.0,supply new,100,940,substitution


## Séparer les features et les labels

In [84]:
# Séparer les features et les labels
X = df.iloc[:, :-1].values  # Prend toutes les colonnes sauf la dernière
y = df.iloc[:, -1].values   # Prend la dernière colonne comme étiquette
# Afficher la taille de X et y
print(f"\n📏 Nombre total d'éléments dans X : {X.shape[0]} lignes, {X.shape[1]} colonnes")
print(f"📏 Nombre total d'éléments dans y : {y.shape[0]} valeurs")


📏 Nombre total d'éléments dans X : 317 lignes, 21 colonnes
📏 Nombre total d'éléments dans y : 317 valeurs


## Séparer les données en train/test

In [85]:
# Fonction pour séparer les données en train/test
def train_test_split(X, y, test_size=0.2):
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    split = int(len(X) * (1 - test_size))
    return X[indices[:split]], X[indices[split:]], y[indices[:split]], y[indices[split:]]

#Séparer les données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Vérifications
print("\n Vérification de la séparation Train/Test :")

# Vérifier la taille des ensembles
print(f"📏 Taille de X_train : {X_train.shape[0]} lignes, {X_train.shape[1]} colonnes")
print(f"📏 Taille de X_test  : {X_test.shape[0]} lignes, {X_test.shape[1]} colonnes")
print(f"📏 Taille de y_train : {y_train.shape[0]} valeurs")
print(f"📏 Taille de y_test  : {y_test.shape[0]} valeurs")

# Vérifier que le nombre total est toujours correct
total_samples = X_train.shape[0] + X_test.shape[0]
if total_samples == X.shape[0]:
    print("La séparation est correcte : le nombre total d'échantillons est préservé.")
else:
    print("Problème : le nombre total d'échantillons ne correspond pas !")

# Afficher quelques exemples de X_train et X_test
print("\n Exemples de X_train (features d'entraînement) :")
print(X_train[:5])  # Afficher les 5 premières lignes de X_train

print("\n Exemples de X_test (features de test) :")
print(X_test[:5])  # Afficher les 5 premières lignes de X_test

print("\n Exemples de y_train (labels d'entraînement) :")
print(y_train[:5])  # Afficher les 5 premiers labels de y_train

print("\n Exemples de y_test (labels de test) :")
print(y_test[:5])  # Afficher les 5 premiers labels de y_test)



 Vérification de la séparation Train/Test :
📏 Taille de X_train : 253 lignes, 21 colonnes
📏 Taille de X_test  : 64 lignes, 21 colonnes
📏 Taille de y_train : 253 valeurs
📏 Taille de y_test  : 64 valeurs
La séparation est correcte : le nombre total d'échantillons est préservé.

 Exemples de X_train (features d'entraînement) :
[[79565123 'HITACHI' 'AAB' 0 0 0 0 1 0 1 0.0 0 0 1 1 150 116 0.0
  'reparable' 0 1629]
 [79520541 'HITACHI' 'BAT' 0 0 0 1 1 1 0 0.0 1 1 1 1 2 2015 0.0
  'reparable' 27 687]
 [79544011 'HITACHI' 'BB' 0 0 0 1 1 0 1 0.0 1 0 1 1 2 1995 0.0
  'reparable' 0 1629]
 [79545622 'HITACHI' 'ADA' 1 0 0 1 1 1 0 0.0 1 1 1 0 15 355 0.0
  'no longer supply new' 0 2189]
 [79520541 'HITACHI' 'CBB' 0 0 0 1 1 1 0 0.0 1 1 1 1 38 2015 0.0
  'reparable' 27 687]]

 Exemples de X_test (features de test) :
[[79544006 'HITACHI' 'AGA' 0 0 1 1 1 1 0 1.0 0 0 0 1 3748 1324 0.0
  'no longer supply new' 0 589]
 [79565123 'HITACHI' 'BAB' 0 0 0 0 1 0 1 0.0 0 0 1 1 4 116 0.0
  'reparable' 0 1629]
 [79

In [86]:
# Fonction pour calculer l'impureté de Gini
def gini_impurity(labels):
    counts = np.bincount(labels)
    probs = counts / len(labels)
    print("Impureté de Gini:")
    print(1 - np.sum(probs ** 2))
    return 1 - np.sum(probs ** 2)

# Fonction pour trouver la meilleure séparation (feature + seuil) dans les données
def best_split(X, y):
    num_samples, num_features = X.shape
    best_gini = float("inf")
    best_feature, best_threshold = None, None

    for feature in range(num_features):
        thresholds = np.unique(X[:, feature])  # Tester tous les seuils possibles
        for threshold in thresholds:
            left_idx = X[:, feature] <= threshold
            right_idx = X[:, feature] > threshold

            if len(y[left_idx]) == 0 or len(y[right_idx]) == 0:
                continue

            left_gini = gini_impurity(y[left_idx])
            right_gini = gini_impurity(y[right_idx])

            gini = (len(left_idx) * left_gini + len(right_idx) * right_gini) / len(y)

            if gini < best_gini:
                best_gini = gini
                best_feature = feature
                best_threshold = threshold

    print(f"Meilleur feature: {best_feature}, Seuil: {best_threshold}\n")
    return best_feature, best_threshold

In [87]:
# Fonction pour construire un arbre de décision
def grow_tree(X, y, depth=0, max_depth=None):
    
    if max_depth is not None and depth >= max_depth:  # Limite de profondeur atteinte
        return Counter(y).most_common(1)[0][0]
    
    feature, threshold = best_split(X, y)
    
    if feature is None:
        return Counter(y).most_common(1)[0][0]

    left_idx = X[:, feature] <= threshold
    right_idx = X[:, feature] > threshold

    left_tree = grow_tree(X[left_idx], y[left_idx], depth + 1, max_depth)
    right_tree = grow_tree(X[right_idx], y[right_idx], depth + 1, max_depth)

    return (feature, threshold, left_tree, right_tree)

In [88]:
# Fonction pour faire une prédiction avec un arbre
def predict_one(x, tree):
    if isinstance(tree, tuple):
        feature, threshold, left_tree, right_tree = tree
        if x[feature] <= threshold:
            return predict_one(x, left_tree)
        else:
            return predict_one(x, right_tree)
    return tree

# Fonction pour prédire sur un ensemble d'exemples
def predict(X, tree):
    return np.array([predict_one(x, tree) for x in X])

In [89]:
# Fonction pour effectuer un échantillonnage bootstrap
def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    indices = np.random.choice(n_samples, n_samples, replace=True)
    return X[indices], y[indices]

In [90]:
# Fonction pour entraîner un modèle Random Forest
def random_forest(X_train, y_train, n_trees=10, max_depth=None):
    trees = []
    
    for _ in range(n_trees):
        X_sample, y_sample = bootstrap_sample(X_train, y_train)
        tree = grow_tree(X_sample, y_sample, max_depth=max_depth)
        trees.append(tree)
    
    return trees

In [91]:
# Fonction pour prédire avec la forêt
def random_forest_predict(X, trees):
    tree_predictions = np.array([predict(X, tree) for tree in trees])
    majority_votes = [Counter(tree_predictions[:, i]).most_common(1)[0][0] for i in range(X.shape[0])]
    return np.array(majority_votes)

# Initialiser et entraîner le modèle Random Forest
trees = random_forest(X_train, y_train, n_trees=10, max_depth=5)

# Prédictions et évaluation
y_pred = random_forest_predict(X_test, trees)
accuracy = np.mean(y_pred == y_test)

# Afficher la précision
print(f"Précision du modèle : {accuracy:.2f}")


TypeError: Cannot cast array data from dtype('O') to dtype('int64') according to the rule 'safe'